In [1]:
import pydot
import matplotlib.pyplot as plt
import time
import math
import random
from sklearn.manifold import MDS
import numpy as np
import plotly.graph_objs as go
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap
from sklearn.metrics import euclidean_distances
from collections import OrderedDict


In [2]:
run 'Step1.ipynb'

In [3]:
run 'Step4.ipynb'

In [4]:
run 'Step6.ipynb'

# Distance Matrix

In [20]:
def adjacent(file,dir = True):
    nodes,edges = get_data(file)[2],get_data(file)[3]
    adj = {int(node):[int(child) for child in child_nodes(edges,node)[1]] for node in nodes}

    if dir == False:
        for node in nodes:
            parents = parent_nodes(edges,node)[1]
            print('node',node,'parents',parents)
            for parent in parents:
                adj[int(node)].append(int(parent))
    for node in adj:
        adj[node] = sorted(adj[node])
    return adj

print(adjacent('Datasets/LesMiserables.dot',dir = False))


def bfs_distance(graph, start):
    visited = set()
    distances = {start: 0}
    queue = [start]

    while queue:
        current_node = queue.pop(0)
        visited.add(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                distances[neighbor] = distances[current_node] + 1
                queue.append(neighbor)

    return distances


def compute_distance_matrix(file,dir = True,p=False):
    graph = adjacent(file,dir)
    nodes = get_data(file)[2]

    distance_matrix = {int(node):bfs_distance(graph, node) for node in graph}

    for node1 in distance_matrix:
        left_nodes = set([int(node) for node in nodes])-set(distance_matrix[node1].keys())
        for node2 in left_nodes:
            distance_matrix[node1][int(node2)] = -1
            
    ordered_dict = dict(OrderedDict(sorted({sub_dict: dict(OrderedDict(sorted(distance_matrix[sub_dict].items()))) for sub_dict in distance_matrix.keys() }.items())))
    ans = [list(v.values()) for v in ordered_dict.values()]
    if p== True:
        print("Similarity matrix:")
        new_nodes = sorted([int(node) for node in nodes])
        print("\t" + "\t".join([str(node) for node in new_nodes]))
        for i in range(len(node_list)):
            print(new_nodes[i], "\t", end="")
            for j in range(len(node_list)):
                print(ans[i][j], "\t", end="")
            print()
    
    return np.array(ans)


#distance_matrix = compute_distance_matrix('Datasets/smallnet.dot',dir=False,p=True)

#distance_matrix = compute_distance_matrix('Datasets/LesMiserables.dot',dir = False,p=True)
#distance_matrix = compute_distance_matrix('Datasets/JazzNetwork.dot',dir = False)
#distance_matrix = compute_distance_matrix('Datasets/LeagueNetwork.dot',p=True)
#print(distance_matrix)
# for node in distance_matrix:
#     print(f"Distances from node {node}: {distance_matrix[node]}")


node 1 parents []
node 2 parents ['1']
node 3 parents ['2']
node 4 parents ['2', '3']
node 5 parents ['2']
node 6 parents ['2']
node 7 parents ['2']
node 8 parents ['2']
node 9 parents ['2']
node 10 parents ['2']
node 11 parents ['2', '3', '4']
node 12 parents ['11']
node 13 parents ['11']
node 14 parents ['11']
node 15 parents ['11']
node 16 parents ['11']
node 24 parents ['11', '13']
node 25 parents ['11', '24']
node 26 parents ['11', '24', '25']
node 27 parents ['11', '25', '26']
node 28 parents ['11', '24', '25', '26', '27']
node 29 parents ['11', '28']
node 30 parents ['11', '24', '28']
node 32 parents ['11', '24', '28']
node 33 parents ['11']
node 34 parents ['11', '28']
node 35 parents ['11', '30']
node 36 parents ['11', '30', '35']
node 37 parents ['11', '30', '35', '36']
node 38 parents ['11', '30', '35', '36', '37']
node 39 parents ['11', '30', '35', '36', '37', '38']
node 44 parents ['11', '27', '28']
node 45 parents ['11', '29']
node 49 parents ['11', '26', '28']
node 50 pa

In [11]:
new = np.array(distance_matrix)
print(new)

[[ 0  5 10 ...  6 21 31]
 [12  0 10 ... 14 12 15]
 [10 15  0 ... 15 12 24]
 ...
 [20 18 22 ...  0  9 16]
 [ 9  6  9 ...  8  0 12]
 [15 11 15 ... 14  5  0]]


In [12]:
print(type(new))
print(type(distance_matrix))

<class 'numpy.ndarray'>
<class 'list'>


In [14]:
print(type(distance_matrix))

<class 'list'>


# Metrics TSNE

In [7]:
from collections import OrderedDict

# Original dictionary
#distance_matrix = distance_matrix 
#new_dict = {sub_dict: dict(OrderedDict(sorted(distance_matrix[sub_dict].items()))) for sub_dict in distance_matrix.keys() }
#for sub_dict in distance_matrix.keys():
    #new_dict[sub_dict] =

print("Similarity matrix:")
new_nodes = sorted([int(node) for node in node_list])
print("\t" + "\t".join([str(node) for node in new_nodes]))
for i in range(len(node_list)):
   print(new_nodes[i], "\t", end="")
   for j in range(len(node_list)):
       print(distance_matrix[i][j], "\t", end="")
   print()

#ordered_dict = dict(OrderedDict(sorted({sub_dict: dict(OrderedDict(sorted(distance_matrix[sub_dict].items()))) for sub_dict in distance_matrix.keys() }.items())))

#print(new_dict)
#print(distance_matrix)
#print(ordered_dict) 


Similarity matrix:


NameError: name 'node_list' is not defined

# T-sne

In [ ]:
def plot_tsne(distances, node_list, edge_list, perplexity=10, learning_rate=300, n_iter=750, interactive=False):
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter, init='pca')
    embeddings = tsne.fit_transform(distances)

    kl_div = tsne.kl_divergence_
    for i in range(n_iter):
        print(f"Iteration {i+1}: KL divergence = {kl_div}")
        if kl_div < 0.05:
            print(f"t-SNE has converged in iteration n. {i+1}")
            break

    # KMeans clustering
    # kmeans = KMeans(n_clusters=8)  # You can adjust the number of clusters
    # labels = kmeans.fit_predict(embeddings)

    plt.figure(figsize=(10, 8))
    #plt.scatter(embeddings[:, 0], embeddings[:, 1], c=labels, cmap='viridis', s=50)
    plt.scatter(embeddings[:, 0], embeddings[:, 1], cmap='viridis', s=50)

    for edge in edge_list:
        u, v = edge
        u_index = node_list.index(u)
        v_index = node_list.index(v)
        plt.plot([embeddings[u_index, 0], embeddings[v_index, 0]], 
                    [embeddings[u_index, 1], embeddings[v_index, 1]], 
                    color='gray', alpha=0.2)

    for i, node_number in enumerate(node_list):
        plt.text(embeddings[i, 0], embeddings[i, 1], node_number, color='black')

    plt.title('t-SNE Visualization of Node Similarities')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.grid(True)
    #plt.colorbar(label='Cluster ID')
    plt.colorbar(label='Similarity')
    plt.show()

# Miserables

In [ ]:
nodes,edges,node_list,edge_list,weights = get_data_step6('Datasets/LesMiserables.dot')

In [ ]:
def compute_similarity_matrix(node_list, edge_list, weights):
    edge_weights = {(u, v): weight for (u, v), weight in weights.items()}

    num_nodes = len(node_list)
    distances = np.zeros((num_nodes, num_nodes))
    for i, source in enumerate(node_list):
        for j, target in enumerate(node_list):
            if i != j:
                if (source, target) in edge_weights:
                    distances[i, j] = edge_weights[(source, target)]
                elif (target, source) in edge_weights:
                    distances[i, j] = edge_weights[(target, source)]
                else:
                    distances[i, j] = np.inf

    similarities = np.zeros((num_nodes, num_nodes))
    for u, v in edge_list:
        i = node_list.index(u)
        j = node_list.index(v)
        similarities[i, j] = 1 / (1 + distances[i, j])
        similarities[j, i] = similarities[i, j]  # Similarity matrix is symmetric

    np.fill_diagonal(similarities, 1) #diagonal 1

    return similarities

similarity_matrix = compute_similarity_matrix(node_list, edge_list, weights)
print(similarity_matrix)
print("Similarity matrix:")
print("\t" + "\t".join(node_list))
for i in range(len(node_list)):
   print(node_list[i], "\t", end="")
   for j in range(len(node_list)):
       print("{:.2f}".format(similarity_matrix[i, j]), "\t", end="")
   print()

similarities = compute_similarity_matrix(node_list, edge_list, weights)


[[1.         0.5        0.         ... 0.         0.         0.        ]
 [0.5        1.         0.11111111 ... 0.         0.         0.        ]
 [0.         0.11111111 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
Similarity matrix:
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	24	25	26	27	28	29	30	32	33	34	35	36	37	38	39	44	45	49	50	52	56	59	65	69	70	71	72	73	17	18	19	20	21	22	23	31	42	43	51	40	41	76	55	46	53	58	63	47	48	60	61	62	64	66	67	74	75	77	57	54	68
1 	1.00 	0.50 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 

In [ ]:
def floyd_warshall(node_list, edge_list, weights):
    num_nodes = len(node_list)
    
    distances = np.full((num_nodes, num_nodes), np.inf)

    np.fill_diagonal(distances, 0)

    for (u, v), weight in weights.items():
        i = node_list.index(u)
        j = node_list.index(v)
        distances[i, j] = weight
        distances[j, i] = weight  # If the graph is undirected

    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                distances[i, j] = min(distances[i, j], distances[i, k] + distances[k, j])

    return distances

distances = floyd_warshall(node_list, edge_list, weights)
print(distances)


In [ ]:
plot_tsne(distances, node_list, edge_list, perplexity=15, learning_rate=300, n_iter=1000, interactive=False)

# Jazz


In [ ]:
nodes,edges,node_list,edge_list,weights = get_data_step6('Datasets/JazzNetwork.dot')

#print(nodes) 
#print(edges)
#print(node_list)
#print(edge_list)
#print(weights)

In [ ]:
def compute_similarity_matrix(node_list, edge_list): #no wweights
    num_nodes = len(node_list)
    
    distances = np.full((num_nodes, num_nodes), np.inf)
    
    for u, v in edge_list:
        i = node_list.index(u)
        j = node_list.index(v)
        distances[i, j] = 1
        distances[j, i] = 1
    
    similarities = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                similarities[i, j] = 1 / (1 + distances[i, j])
    
    np.fill_diagonal(similarities, 1)
    
    return similarities

similarities = compute_similarity_matrix(node_list, edge_list)

#print("Similarity matrix:")
#print("\t" + "\t".join(node_list))
#for i in range(len(node_list)):
#    print(node_list[i], "\t", end="")
#    for j in range(len(node_list)):
#        print("{:.2f}".format(similarities[i, j]), "\t", end="")
#    print()



In [ ]:
plot_tsne(similarities, node_list, edge_list, perplexity=15, learning_rate=250, n_iter=1500, interactive=False)

# League

In [ ]:
nodes,edges,node_list,edge_list,weights = get_data_league('Datasets/LeagueNetwork.dot')

#print(nodes) 
#print(edges)
#print(node_list)
#print(edge_list)
#print(weights)

In [ ]:
def compute_similarity_matrix_league(node_list, edge_list, weights):
    edge_weights = {(u, v): weight for (u, v), weight in weights.items()}

    num_nodes = len(node_list)
    distances = np.zeros((num_nodes, num_nodes))
    
    for i, source in enumerate(node_list):
        for j, target in enumerate(node_list):
            if i != j:
                if (source, target) in edge_weights:
                    distances[i, j] = edge_weights[(source, target)]
                else:
                    distances[i, j] = -edge_weights[(target, source)] if (target, source) in edge_weights else np.inf

    similarities = np.zeros((num_nodes, num_nodes))
    for u, v in edge_list:
        i = node_list.index(u)
        j = node_list.index(v)
        similarities[i, j] = 1 / (1 + abs(distances[i, j])) if distances[i, j] != np.inf else 0 
        similarities[j, i] = similarities[i, j]  

    np.fill_diagonal(similarities, 1)  # Diagonal set to 1

    return similarities

similarities = compute_similarity_matrix_league(node_list, edge_list, weights)

#print("Similarity matrix:")
#print("\t" + "\t".join(node_list))
#for i in range(len(node_list)):
#    print(node_list[i], "\t", end="")
#    for j in range(len(node_list)):
#        print("{:.2f}".format(similarities[i, j]), "\t", end="")
#    print()

In [ ]:
plot_tsne(similarities, node_list, edge_list, perplexity=10, learning_rate=200, n_iter=1000, interactive=False)

# Testing

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def compute_continuity(similarities, distances_Y, k=5):
    """
    Compute continuity metric for t-SNE projection.

    Parameters:
    - similarities: Similarity matrix for the high-dimensional data.
    - distances_Y: Euclidean distances for the t-SNE projected points in 2D space.
    - k: Number of neighbors to consider.

    Returns:
    - Continuity score.
    """
    n = len(similarities)

    nn_orig = similarities.argsort()
    nn_proj = distances_Y.argsort()

    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        V = np.setdiff1d(knn_orig[i], knn_proj[i])

        sum_j = 0
        for j in range(V.shape[0]):
            sum_j += np.where(nn_proj[i] == V[j])[0] - k

        sum_i += sum_j

    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)).squeeze())

def compute_trustworthiness(similarities, distances_Y, k=5):
    """
    Compute trustworthiness metric for t-SNE projection.

    Parameters:
    - similarities: Similarity matrix for the high-dimensional data.
    - distances_Y: Euclidean distances for the t-SNE projected points in 2D space.
    - k: Number of neighbors to consider.

    Returns:
    - Trustworthiness score.
    """
    n = len(similarities)

    nn_orig = similarities.argsort()
    nn_proj = distances_Y.argsort()

    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        U = np.setdiff1d(knn_proj[i], knn_orig[i])

        sum_j = 0
        for j in range(U.shape[0]):
            sum_j += np.where(nn_orig[i] == U[j])[0] - k

        sum_i += sum_j

    trustworthiness_score = float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)).squeeze())
    return trustworthiness_score

def compute_stress(similarities, distances_Y):
    
    #print(f"similarities: {similarities}")
    #print(f"distances_Y: {distances_Y}")
    #n = len(similarities)
    
    # Create a mask to ignore the diagonal and only consider the lower triangular part
    #mask = np.tri(n, k=-1)
    stress = np.sum((similarities - distances_Y)**2) / np.sum(similarities**2)
    #stress = np.sum((similarities - distances_Y)**2 * mask) / np.sum(similarities**2 * mask)

    return stress

def compute_shepard(distances_highD, distances_lowD):
    
    shepard_values = []
    plt.scatter(distances_highD, distances_lowD)

    return shepard_values

#def metric_pq_shepard_diagram_correlation(id_run, dataset_name):
#    global DISTANCES
#
#    D_high = np.load(DISTANCES['D_high_list'], mmap_mode='c')
#    D_low = np.load(DISTANCES[id_run]['D_low_list'], mmap_mode='c')
#
#    return stats.spearmanr(D_high, D_low)[0]

def floyd_warshall(node_list, edge_list, weights):
    num_nodes = len(node_list)
    
    distances = np.full((num_nodes, num_nodes), np.inf)

    np.fill_diagonal(distances, 0)

    for (u, v), weight in weights.items():
        i = node_list.index(u)
        j = node_list.index(v)
        distances[i, j] = weight
        distances[j, i] = weight  # If the graph is undirected

    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                distances[i, j] = min(distances[i, j], distances[i, k] + distances[k, j])

    return distances

distances = floyd_warshall(node_list, edge_list, weights)
print(distances)

def plot_tsne_testing(distances, node_list, edge_list, perplexity=10, learning_rate=300, n_iter=750):
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter, init='pca')
    embeddings = tsne.fit_transform(distances)
    distances_Y = euclidean_distances(embeddings)
    #kl_div = tsne.kl_divergence_
    #for i in range(n_iter):
    #    print(f"Iteration {i+1}: KL divergence = {kl_div}")
    #    if kl_div < 0.05:
    #        print(f"t-SNE has converged in iteration n. {i+1}")
    #        break

    continuity_score = compute_continuity(similarities, distances_Y)
    trustworthiness_score = compute_trustworthiness(similarities, distances_Y)
    stress = compute_stress(similarities, distances_Y)
    shepard_values = compute_shepard(similarities.flatten(), distances_Y.flatten())

    print(f"Continuity {continuity_score}")
    print(f"Trustworthiness {trustworthiness_score}")
    print(f"Stress {stress}")

    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], cmap='viridis', s=50)

    for edge in edge_list:
        u, v = edge
        u_index = node_list.index(u)
        v_index = node_list.index(v)
        plt.plot([embeddings[u_index, 0], embeddings[v_index, 0]], 
                 [embeddings[u_index, 1], embeddings[v_index, 1]], 
                 color='gray', alpha=0.2)

    for i, node_number in enumerate(node_list):
        plt.text(embeddings[i, 0], embeddings[i, 1], node_number, color='black')

    plt.title('t-SNE Visualization of Node Similarities')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    #plt.grid(True)
    #plt.colorbar(label='Similarity')
    plt.show()

nodes, edges, node_list, edge_list, weights = get_data_step6('Datasets/LesMiserables.dot')
similarities = compute_similarity_matrix(node_list, edge_list, weights)
plot_tsne_testing(similarities, node_list, edge_list, perplexity=10,learning_rate=300, n_iter=1000)


In [ ]:
similarities.flatten()